In [2]:
!pip install torchpack

  Using cached torchpack-0.3.1-py3-none-any.whl (34 kB)
  Using cached multimethod-1.8-py3-none-any.whl (9.8 kB)
     |████████████████████████████████| 58 kB 63 kB/s eta 0:00:01
     |████████████████████████████████| 4.5 MB 102 kB/s eta 0:00:01
  Using cached tensorpack-0.11-py2.py3-none-any.whl (296 kB)
     |████████████████████████████████| 322 kB 50 kB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [1]:
import argparse
import copy
import os
import warnings

import mmcv
from mmcv.utils import build_from_cfg
import torch
from torchpack.utils.config import configs
from torchpack import distributed as dist
from mmcv import Config, DictAction
from mmcv.cnn import fuse_conv_bn
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmcv.runner import get_dist_info, init_dist, load_checkpoint, wrap_fp16_model
from mmdet3d.apis import single_gpu_test
from mmdet3d.datasets import build_dataloader, build_dataset
from mmdet3d.models import build_model
from mmdet.apis import multi_gpu_test, set_random_seed
from mmdet.datasets import replace_ImageToTensor
from mmdet3d.utils import recursive_eval

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("config", metavar="FILE")
parser.add_argument("--mode", type=str, default="pred", choices=["gt", "pred"])
parser.add_argument("--checkpoint", type=str, default=None)
parser.add_argument("--split", type=str, default="val", choices=["train", "val"])
parser.add_argument("--bbox-classes", nargs="+", type=int, default=None)
parser.add_argument("--bbox-score", type=float, default=None)
parser.add_argument("--map-score", type=float, default=0.5)
parser.add_argument("--out-dir", type=str, default="viz")
parser.add_argument("--fp16", action="store_true")
args, opts = parser.parse_known_args(['configs/nuscenes/det/transfusion/secfpn/camera+lidar/swint_v0p075/convfuser.yaml', '--checkpoint', 'pretrained/bevfusion-det.pth', '--fp16'])

In [3]:
#configs.update(opts)
config_file = 'configs/nuscenes/det/transfusion/secfpn/camera+lidar/swint_v0p075/convfuser.yaml'
configs.load(config_file, recursive=True)
checkpoint = 'pretrained/bevfusion-det.pth'
cfg = Config(recursive_eval(configs), filename=config_file)

In [4]:
cfg['data']['test']['pipeline'][4]['final_dim'] = [128, 360]
cfg.model.encoders['camera'].vtransform['image_size'] = [128,360]
cfg.model.encoders['camera'].vtransform['feature_size'] = [16, 45]

In [5]:
torch.backends.cudnn.benchmark = cfg.cudnn_benchmark
torch.cuda.set_device(dist.local_rank())

In [6]:
dataset = build_dataset(cfg.data['test'])
dataflow = build_dataloader(
    dataset,
    samples_per_gpu=1,
    workers_per_gpu=2,
    dist=False,
    shuffle=False,
)

In [7]:
model = build_model(cfg.model, test_cfg=cfg.get("test_cfg"))
#wrap_fp16_model(model)
sd = load_checkpoint(model, checkpoint, map_location="cpu")
model = model.to('cuda')
"""model = MMDataParallel(
            model.cuda(),
            device_ids=[torch.cuda.current_device()],
        )"""

/usr/local/lib/python3.8/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2022-08-11 18:15:17,524 - mmdet - INFO - load checkpoint from local path: /workspace/catkin_ws_devel/src/conav_perception/conav_bev_detection/src/bevfusion/pretrained/swin_tiny_patch4_window7_224.pth


load checkpoint from local path: pretrained/bevfusion-det.pth
The model and loaded state dict do not match exactly

size mismatch for encoders.camera.vtransform.frustum: copying a param with shape torch.Size([118, 32, 88, 3]) from checkpoint, the shape in current model is torch.Size([118, 16, 45, 3]).


'model = MMDataParallel(\n            model.cuda(),\n            device_ids=[torch.cuda.current_device()],\n        )'

In [30]:
sd['state_dict']['encoders.camera.vtransform.frustum']


tensor([[[[  0.0000,   0.0000,   1.0000],
          [  8.0805,   0.0000,   1.0000],
          [ 16.1609,   0.0000,   1.0000],
          ...,
          [686.8391,   0.0000,   1.0000],
          [694.9196,   0.0000,   1.0000],
          [703.0000,   0.0000,   1.0000]],

         [[  0.0000,   8.2258,   1.0000],
          [  8.0805,   8.2258,   1.0000],
          [ 16.1609,   8.2258,   1.0000],
          ...,
          [686.8391,   8.2258,   1.0000],
          [694.9196,   8.2258,   1.0000],
          [703.0000,   8.2258,   1.0000]],

         [[  0.0000,  16.4516,   1.0000],
          [  8.0805,  16.4516,   1.0000],
          [ 16.1609,  16.4516,   1.0000],
          ...,
          [686.8391,  16.4516,   1.0000],
          [694.9196,  16.4516,   1.0000],
          [703.0000,  16.4516,   1.0000]],

         ...,

         [[  0.0000, 238.5484,   1.0000],
          [  8.0805, 238.5484,   1.0000],
          [ 16.1609, 238.5484,   1.0000],
          ...,
          [686.8391, 238.5484,   1.00

In [23]:
model.encoders.camera.vtransform.frustum

Parameter containing:
tensor([[[[  0.0000,   0.0000,   1.0000],
          [  8.1591,   0.0000,   1.0000],
          [ 16.3182,   0.0000,   1.0000],
          ...,
          [342.6818,   0.0000,   1.0000],
          [350.8409,   0.0000,   1.0000],
          [359.0000,   0.0000,   1.0000]],

         [[  0.0000,   8.4667,   1.0000],
          [  8.1591,   8.4667,   1.0000],
          [ 16.3182,   8.4667,   1.0000],
          ...,
          [342.6818,   8.4667,   1.0000],
          [350.8409,   8.4667,   1.0000],
          [359.0000,   8.4667,   1.0000]],

         [[  0.0000,  16.9333,   1.0000],
          [  8.1591,  16.9333,   1.0000],
          [ 16.3182,  16.9333,   1.0000],
          ...,
          [342.6818,  16.9333,   1.0000],
          [350.8409,  16.9333,   1.0000],
          [359.0000,  16.9333,   1.0000]],

         ...,

         [[  0.0000, 110.0667,   1.0000],
          [  8.1591, 110.0667,   1.0000],
          [ 16.3182, 110.0667,   1.0000],
          ...,
          [342.

In [26]:
model.encoders.camera.vtransform

DepthLSSTransform(
  (dtransform): Sequential(
    (0): Conv2d(1, 8, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(8, 32, kernel_size=(5, 5), stride=(4, 4), padding=(2, 2))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
  )
  (depthnet): Sequential(
    (0): Conv2d(320, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (

In [8]:
model = model.eval()

In [9]:
from tqdm import tqdm
for data in tqdm(dataflow):
    """for key in data.keys():
            if isinstance(data[key].data[0], torch.Tensor):
                if not key.startswith('gt'):
                    print(key, data[key].data[0].shape)
                    data[key].data[0] = data[key].data[0][:,:4]"""
    break

  0%|                                                    | 0/81 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/nuscenes/map_expansion/map_api.py:1823: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
/usr/local/lib/python3.8/dist-packages/nuscenes/map_expansion/map_api.py:1824: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  interiors = [int_coords(pi.coords) for poly in polygons for pi in poly.interiors]
/usr/local/lib/python3.8/dist-packages/nuscenes/map_expansion/map_api.py:1823: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` prope

In [10]:
with torch.no_grad():
    op = model(**data)

AttributeError: DataContainer has no attribute size for type <class 'list'>

In [14]:
op

{'stats/object/loss_heatmap': tensor(4.2321, device='cuda:0'),
 'stats/object/layer_-1_loss_cls': tensor(0.5990, device='cuda:0'),
 'stats/object/layer_-1_loss_bbox': tensor(6.9312, device='cuda:0'),
 'stats/object/matched_ious': tensor(0.0440, device='cuda:0')}

In [14]:
model.module.encoders['lidar'].backbone.encoder_layers.encoder_layer1[0].conv1.bfloat16

<bound method Module.bfloat16 of SubMConv3d()>

In [12]:
model = build_model(cfg.model)
sd = load_checkpoint(model, checkpoint, map_location="cpu")
#wrap_fp16_model(model)
#model = model.to('cuda')
model = MMDataParallel(
            model.cuda(),
            device_ids=[torch.cuda.current_device()],
        )

2022-08-08 17:57:32,796 - mmdet - INFO - load checkpoint from local path: pretrained/swin_tiny_patch4_window7_224.pth


load checkpoint from local path: pretrained/bevfusion-det.pth
The model and loaded state dict do not match exactly

size mismatch for encoders.camera.vtransform.frustum: copying a param with shape torch.Size([118, 32, 88, 3]) from checkpoint, the shape in current model is torch.Size([118, 16, 45, 3]).


In [15]:
from tqdm import tqdm
for data in tqdm(dataflow):
    """for key in data.keys():
            if isinstance(data[key].data[0], torch.Tensor):
                if not key.startswith('gt'):
                    print(key, data[key].data[0].shape)
                    data[key].data[0] = data[key].data[0][:,:4]"""
    break

  0%|                                                                                                                                                                                  | 0/81 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/nuscenes/map_expansion/map_api.py:1823: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
/usr/local/lib/python3.8/dist-packages/nuscenes/map_expansion/map_api.py:1824: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  interiors = [int_coords(pi.coords) for poly in polygons for pi in poly.interiors]
/usr/local/lib/python3.8/dist-packages/nuscenes/map_expansion/map_api.py:1823: ShapelyDe

In [16]:
model = model.eval()
with torch.inference_mode():
    op = model(**data)

lidar featuress extracted
[128, 360]
camera featuress extracted


In [19]:
op[0]['boxes_3d'].center

tensor([[-5.7437e+00,  6.3812e+00, -2.5997e+00],
        [-5.1823e+00,  8.8124e+00, -2.2661e+00],
        [ 6.1482e-01, -1.9569e+01, -1.4872e+00],
        [-4.1648e-01, -1.7927e+01, -2.4257e+00],
        [-9.5414e+00,  7.8801e-01, -1.1570e+00],
        [-9.5050e+00,  1.9483e+00, -1.2074e+00],
        [-6.4013e+00,  6.3365e+00, -1.7546e+00],
        [-6.1581e+00,  3.2099e+01, -2.3964e-02],
        [-6.2377e+00,  5.3622e+01,  2.8905e-01],
        [ 9.0829e+00, -5.6738e+00,  5.9229e-02],
        [ 5.8179e+00,  1.7086e+01, -1.1511e+00],
        [-5.7589e+00, -5.6633e+00, -2.9378e+00],
        [-7.3706e-01, -1.7831e+01, -1.6173e+00],
        [-1.0719e+01, -5.5927e+00, -1.0708e+00],
        [ 5.1826e+00, -6.4060e+00, -9.4386e-01],
        [ 5.1371e+00, -5.1961e+00, -9.1201e-01],
        [-4.7206e+00,  8.8437e+00, -1.8410e+00],
        [ 5.1229e+00, -5.1577e+00, -9.7169e-01],
        [ 2.8162e+00,  2.4307e+00, -1.9742e+00],
        [ 1.0505e+00,  6.7921e-01, -6.3047e-01],
        [-2.5490e+00

In [17]:
op

[{'boxes_3d': LiDARInstance3DBoxes(
      tensor([[-5.7437e+00,  6.3812e+00, -2.5997e+00,  ..., -2.3723e+00,
            1.0003e-01,  2.7994e-01],
          [-5.1823e+00,  8.8124e+00, -2.2661e+00,  ..., -2.4272e+00,
            1.6967e-02,  1.6725e-01],
          [ 6.1482e-01, -1.9569e+01, -1.4872e+00,  ..., -2.4025e+00,
           -1.7416e-01, -3.4264e-01],
          ...,
          [-1.1067e+01, -2.7728e-01, -1.4883e+00,  ...,  2.8130e+00,
           -3.6019e-01, -2.5236e-01],
          [-8.6175e+00, -1.8883e+01, -1.3853e+00,  ..., -2.1663e+00,
           -5.1443e-04, -2.3917e-03],
          [-1.1542e+01,  7.4380e+00, -4.0616e-01,  ...,  2.2316e+00,
           -4.8096e-04, -2.8448e-03]])),
  'scores_3d': tensor([5.5366e-03, 1.8386e-03, 4.6990e-07, 1.0725e-03, 5.6083e-04, 5.7997e-04,
          1.2708e-03, 8.6969e-04, 6.5436e-04, 1.0384e-05, 4.8818e-04, 5.8379e-04,
          3.6558e-04, 5.0699e-04, 5.3108e-04, 3.4859e-04, 3.6513e-04, 2.8120e-04,
          1.5266e-04, 4.6069e-07, 3.0434e

In [15]:
from tqdm import tqdm
for data in tqdm(dataflow):
        metas = data["metas"].data[0][0]
        name = "{}-{}".format(metas["timestamp"], metas["token"])
        for key in data.keys():
            data[key] = data[key].data[0]
            if key =='points':
                data[key][0] = data[key][0].to('cuda')
            if isinstance(data[key], torch.Tensor):
                data[key] = data[key].to('cuda')
            """if key.startswith('gt'):
                data[key] = None"""
        
        break

  0%|                                                    | 0/81 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/nuscenes/map_expansion/map_api.py:1823: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
/usr/local/lib/python3.8/dist-packages/nuscenes/map_expansion/map_api.py:1824: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  interiors = [int_coords(pi.coords) for poly in polygons for pi in poly.interiors]
/usr/local/lib/python3.8/dist-packages/nuscenes/map_expansion/map_api.py:1823: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` prope

In [21]:
with torch.inference_mode():
    outputs = model(**data)

[128, 360]
camera featuress extracted
lidar featuress extracted


In [31]:
from mmdet3d.core import LiDARInstance3DBoxes
bboxes = outputs[0]["boxes_3d"].tensor.numpy()
scores = outputs[0]["scores_3d"].numpy()
labels = outputs[0]["labels_3d"].numpy()
bbox_score = 0.3
if args.bbox_classes is not None:
    indices = np.isin(labels, args.bbox_classes)
    bboxes = bboxes[indices]
    scores = scores[indices]
    labels = labels[indices]

if bbox_score is not None:
    indices = scores >= bbox_score
    bboxes = bboxes[indices]
    scores = scores[indices]
    labels = labels[indices]

bboxes[..., 2] -= bboxes[..., 5] / 2
bboxes = LiDARInstance3DBoxes(bboxes, box_dim=9)

In [30]:
bbox_score

NameError: name 'bbox_score' is not defined

In [23]:
bboxes

LiDARInstance3DBoxes(
    tensor([[ 5.6580e+00, -2.2935e-01, -1.6376e+00,  ..., -2.4811e+00,
          1.4811e-01,  1.9587e-01],
        [ 6.4855e+00, -1.5290e-01, -1.5548e+00,  ...,  3.1097e+00,
          1.5066e-03, -1.4600e-02],
        [ 7.5322e+00,  1.7415e+01, -1.1808e+00,  ...,  8.2848e-04,
         -2.8021e-02, -4.4113e-01],
        ...,
        [ 2.3820e+01,  3.8223e+01, -4.2124e-01,  ..., -1.7480e+00,
          5.2935e-03,  6.3262e-03],
        [-4.5935e+01,  1.5636e+01, -2.3868e+00,  ..., -1.6260e+00,
          1.2399e-01, -2.0970e-02],
        [ 4.4504e+00, -3.9212e+01, -3.2324e+00,  ...,  1.3137e+00,
          7.0493e-06,  1.1545e-06]]))

In [24]:
data['points']

[tensor([[-3.1226e+00, -4.2449e-01, -1.8768e+00,  4.0000e+00,  0.0000e+00],
         [-3.2819e+00, -4.2313e-01, -1.8688e+00,  5.0000e+00,  0.0000e+00],
         [-3.4933e+00, -4.2222e-01, -1.8821e+00,  6.0000e+00,  0.0000e+00],
         ...,
         [-7.9269e+00, -1.4184e-03, -1.6849e+00,  3.4000e+01,  0.0000e+00],
         [-8.2272e+00,  1.4265e-03, -1.5501e+00,  2.0000e+01,  0.0000e+00],
         [-8.5262e+00, -3.2753e-02, -1.1983e+00,  1.0000e+02,  0.0000e+00]],
        device='cuda:0')]

In [32]:
from mmdet3d.core.utils import visualize_camera, visualize_lidar
lidar = data["points"][0].to('cpu').numpy()
visualize_lidar(
    os.path.join("temp.png"),
    lidar,
    bboxes=bboxes,
    labels=labels,
    xlim=[cfg.point_cloud_range[d] for d in [0, 3]],
    ylim=[cfg.point_cloud_range[d] for d in [1, 4]],
    classes=cfg.object_classes,
)

In [ ]:
visualize_camera(
    os.path.join("temp_cam.png"),
    lidar,
    bboxes=bboxes,
    labels=labels,
    xlim=[cfg.point_cloud_range[d] for d in [0, 3]],
    ylim=[cfg.point_cloud_range[d] for d in [1, 4]],
    classes=cfg.object_classes,
)

In [56]:
x  = model.encoders['camera']['backbone'](data['img'][0])
x = model.encoders['camera']['neck'](x)
for i in range(len(x)):
    print(x[i].shape)
#feature size  = (256, 16, 45)
x = x[0]

torch.Size([6, 256, 16, 45])
torch.Size([6, 256, 8, 23])


In [57]:
x = x.unsqueeze(0)
cam_features = model.encoders["camera"]["vtransform"](x, data['points'],
        data['camera2ego'],
        data['lidar2ego'],
        data['lidar2camera'],
        data['lidar2image'],
        data['camera_intrinsics'],
        data['img_aug_matrix'],
        data['lidar_aug_matrix'],
        data['metas'])

[128, 360]


In [47]:
model.encoders["camera"]["vtransform"].feature_size

[32, 88]

In [12]:
model = model.to('cuda').half()

#model = model.eval()

In [8]:
feats, coords, sizes = model.encoders['lidar'].voxelize(data['points'][0])

In [11]:
data['points'][0].shape

torch.Size([207998, 5])

In [9]:
batch_size = coords[-1, 0] + 1
x = model.encoders["lidar"]["backbone"](feats, coords, batch_size, sizes=sizes)

RuntimeError: /workspace/catkin_ws_devel/src/conav_perception/conav_bev_detection/src/bevfusion/mmdet3d/ops/spconv/include/spconv/spconv_ops.h 40
kernelSize.size() == coorDim assert faild. error

In [11]:
feats, coords, sizes = model.encoders['lidar'].voxelize.half()(data['points'][0].half())

RuntimeError: "hard_voxelize_kernel" not implemented for 'Half'

In [15]:
feats.shape, coords.shape, sizes.shape

(torch.Size([12677, 10, 5]), torch.Size([12677, 3]), torch.Size([12677]))

In [17]:
from tqdm import tqdm
for data in tqdm(dataflow):
        metas = data["metas"].data[0][0]
        name = "{}-{}".format(metas["timestamp"], metas["token"])
        for key in data.keys():
            data[key] = data[key].data[0]
            if key =='points':
                data[key][0] = data[key][0].to('cuda').half()
            if isinstance(data[key], torch.Tensor):
                data[key] = data[key].to('cuda')[:,:4].half()
            if key.startswith('gt'):
                data[key] = None
        with torch.inference_mode():
            outputs = model(**data)
        break

  0%|                                                    | 0/81 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/nuscenes/map_expansion/map_api.py:1823: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
/usr/local/lib/python3.8/dist-packages/nuscenes/map_expansion/map_api.py:1824: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  interiors = [int_coords(pi.coords) for poly in polygons for pi in poly.interiors]
/usr/local/lib/python3.8/dist-packages/nuscenes/map_expansion/map_api.py:1823: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` prope

RuntimeError: expected scalar type Float but found Half

In [18]:
data

{'img': tensor([[[[[ 0.2649,  0.2649,  0.3335,  ..., -0.7798, -0.7627, -0.8481],
            [ 0.2649,  0.2820,  0.2820,  ..., -0.8652, -0.8652, -0.9683],
            [ 0.2307,  0.2478,  0.2478,  ..., -0.9336, -0.9336, -0.9683],
            ...,
            [-0.2488, -0.2317, -0.2146,  ..., -0.0262, -0.1289, -0.1461],
            [-0.2488, -0.2488, -0.2659,  ..., -0.0433, -0.1461, -0.1803],
            [-0.2659, -0.2830, -0.2830,  ...,  0.0080, -0.0262, -0.0947]],
 
           [[-0.6353, -0.6177, -0.6177,  ..., -1.0381, -1.0205, -1.0908],
            [-0.6704, -0.6528, -0.6353,  ..., -1.0557, -1.0381, -1.0908],
            [-0.6875, -0.6875, -0.6704,  ..., -1.1250, -1.1250, -1.1602],
            ...,
            [-0.3726, -0.3550, -0.3376,  ..., -0.1449, -0.2500, -0.2676],
            [-0.3726, -0.3726, -0.3901,  ..., -0.1625, -0.2676, -0.3025],
            [-0.3901, -0.4075, -0.4075,  ..., -0.1099, -0.1449, -0.2150]],
 
           [[-0.8657, -0.8833, -0.8481,  ..., -1.2666, -1.2490, -

In [16]:
from mmcv.ops.voxelize import Voxelization, voxelization


True

In [9]:
from copy import deepcopy
data2 = deepcopy(data)

In [10]:
data = deepcopy(data2)

In [20]:
data['camera2ego'].shape

torch.Size([1, 6, 4, 4])

In [14]:
data['gt_bboxes_3d'] = None
data['gt_bboxes_3d'] = None
data['gt_masks_bev'] = None


In [10]:
for sensor in model.encoders:
    print(sensor)

camera
lidar


In [12]:
outputs = model(**data)

[256, 704]
B, N  1 6


RuntimeError: CUDA out of memory. Tried to allocate 560.00 MiB (GPU 0; 7.80 GiB total capacity; 4.43 GiB already allocated; 547.25 MiB free; 4.62 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [23]:
print(data['img'].data[0][0].shape)
x = model.encoders['camera']['backbone'](data['img'].data[0][0].to('cuda'))

torch.Size([6, 3, 256, 704])


In [24]:
for i in range(len(x)):
    print(x[i].shape)

torch.Size([6, 192, 32, 88])
torch.Size([6, 384, 16, 44])
torch.Size([6, 768, 8, 22])


In [26]:
model.encoders['camera']['neck']

GeneralizedLSSFPN(
  (lateral_convs): ModuleList(
    (0): ConvModule(
      (conv): Conv2d(448, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (1): ConvModule(
      (conv): Conv2d(1152, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
  )
  (fpn_convs): ModuleList(
    (0): ConvModule(
      (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (1): ConvModule(
      (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [27]:
x = model.encoders['camera']['neck'](x)
for i in range(len(x)):
    print(x[i].shape)

torch.Size([6, 256, 32, 88])
torch.Size([6, 256, 16, 44])
